In [1]:
import numpy as np
import scipy
import glob
from math import pi
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def read_frame(filedesc, bins):
    for i in range(2):
        comment = filedesc.readline()

    grid = np.zeros((bins,2),float)
    value = np.zeros(bins,dtype=complex)
    
    for i in range(bins):
        #blank = filedesc.readline()
        line = filedesc.readline().split();
        grid[i,0] = line[0]
        grid[i,1] = line[1]
        value[i] = np.float(line[2])+1j*np.float(line[3])
        
    return [grid, value]

In [3]:
def gammak(X, gx, gy):
    return gx*X[0]**2.+gy*X[1]**2.

In [4]:
# system properties
surfacearea = 73.059404*71.033600
hkbt = 0.60/2.

# parameters for the CFM analysis
kcut = 0.65
wavelengthcut = 2*pi/kcut
binv = [20, 20]
bins = binv[0]*binv[1]-1
print("cutoff wavelength:", wavelengthcut)
# the fluctuation here is the A_s and A_l from the bulk phases
# As it shows up like a white noise, and is de facto constant at all wavelengthes
# it can be treated as a constant for our LJ system
# However, there's no clearcut reason why this should be a constant, so always check and try to use the actual values
fluctuation = 0.00009

cutoff wavelength: 9.66643893412244


In [5]:
# collect all the filenames
filelist = glob.glob('./ft-files/avgaksqr-*.dat')
print(filelist)

['./ft-files/avgaksqr-9.dat', './ft-files/avgaksqr-7.dat', './ft-files/avgaksqr-10.dat', './ft-files/avgaksqr-3.dat', './ft-files/avgaksqr-2.dat', './ft-files/avgaksqr-4.dat', './ft-files/avgaksqr-6.dat', './ft-files/avgaksqr-5.dat', './ft-files/avgaksqr-8.dat', './ft-files/avgaksqr-1.dat']


In [6]:
nframe = 0
parameters = []
for ifile in filelist:
    #print("Reading file:", ifile)
    traj = open(ifile,"r")
    [ kgrid, ak ] = read_frame(traj, bins)
    nframe += 1
    
    n=0
    X = []
    y = []
    for i in range(len(kgrid)):
        if(kgrid[i,0]<kcut and kgrid[i,1]<kcut):
            X = np.append(X,[kgrid[i,0],kgrid[i,1]])
            y = np.append(y,hkbt/((ak[i].real-fluctuation)*surfacearea)*2.)
            X = np.append(X,[kgrid[i,0],kgrid[i,1]])
            y = np.append(y,hkbt/((ak[i].imag-fluctuation)*surfacearea)*2.)
            n+=2
    #print(np.shape(X),n)
    popt, pcov = curve_fit(gammak, np.reshape(X,(n,2)).T, y)
    print(popt)
    parameters = np.append(parameters, popt)

[ 0.45691918  0.30183245]
[ 0.45471992  0.29533638]
[ 0.45662783  0.28673131]
[ 0.45606692  0.28071275]
[ 0.44859117  0.29219405]
[ 0.45315228  0.28937963]
[ 0.45693889  0.28400056]
[ 0.46209608  0.28425554]
[ 0.45028027  0.29183029]
[ 0.46718711  0.28255179]


In [7]:
gest = np.reshape(parameters,(nframe,2))
g1est=[np.mean(gest[:,0]),np.std(gest[:,0])/np.sqrt(nframe-1)]
print(g1est)
g2est=[np.mean(gest[:,1]),np.std(gest[:,1])/np.sqrt(nframe-1)]
print(g2est)

[0.45625796484822551, 0.0017033539904307523]
[0.28888247474456907, 0.0020708273835056382]
